In [34]:
import sys

app = "/app"
if app not in sys.path:
    sys.path.append(app)
sys.path

['/app/notebooks',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/opt/venv/lib/python3.10/site-packages',
 '/app',
 '/tmp/tmpmt5qg72t']

In [35]:
!pwd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/app/notebooks


In [36]:
!ls /data/emboj_abstracts/train

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


examples.txt


In [37]:
from datasets import load_dataset

dataset = load_dataset(
    "text",
    data_dir="/data/emboj_abstracts/",
    data_files={'train': 'train/examples.txt', 'test': 'test/examples.txt'},
)
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 12881
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1802
    })
})

In [38]:
MODEL_NAME = "roberta-base"

In [39]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"tokenizer vocab size: {tokenizer.vocab_size}")
print(f"max length: {tokenizer.max_model_input_sizes[MODEL_NAME]}")

tokenizer vocab size: 50265
max length: 512


In [40]:
# tokenize the dataset
def tokenization(examples):
    return tokenizer(
    examples["text"],
        max_length=64, #tokenizer.max_model_input_sizes[MODEL_NAME],
        truncation=True,
        return_special_tokens_mask=True,
    )
tokenized = dataset.map(tokenization, batched=True)
tokenized

Map:   0%|          | 0/1802 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 12881
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 1802
    })
})

In [41]:
tokenized = tokenized.remove_columns(["text"])
tokenized

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 12881
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 1802
    })
})

In [42]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer,
    mlm=True,
    mlm_probability=0.15
)

In [53]:
from transformers import (
    # RobertaForMaskedLM,
    AutoConfig
)
from src.models.modeling_dendroberta import RobertaForMaskedLM

In [54]:
config = AutoConfig.from_pretrained(MODEL_NAME)
config

RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.36.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [55]:
config.num_hidden_layers=8
# config.intermediate_size=128
config.max_position_embeddings=66

In [56]:
model = RobertaForMaskedLM(config=config)
model_size = sum(t.numel() for t in model.parameters())
print(f"Model size: {model_size/1000**2:.1f}M parameters")

Model size: 96.0M parameters


In [57]:
from transformers import Trainer, TrainingArguments
from datetime import datetime

args = TrainingArguments(
    output_dir="/models",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="steps",
    eval_steps=400,
    prediction_loss_only=True,
    logging_steps=1000,
    gradient_accumulation_steps=1,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=False,
    push_to_hub=False,
    logging_dir = f"/runs/lm-dendroberta-{datetime.now().isoformat().replace(':','-')}"
)

In [58]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized["test"],
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer.train()

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# launch tensorboard
%tensorboard --logdir /runs